In [7]:
%load_ext autoreload
%autoreload 2

# Dependencies

In [18]:
import datasets_handler
import evaluation_metrics
from zeroberto import ZeroBERTo
import gc
import pandas as pd
from datasets import load_dataset, Dataset


In [9]:
# df = pd.read_csv('/Users/alealcoforado/Documents/Projetos/Datasets/folhauol/folhauol_clean_df_articles.csv',nrows=4000)
# df.to_csv('/Users/alealcoforado/Documents/Projetos/Datasets/folhauol/folhauol_4000examples_clean_df_articles.csv')

In [14]:
which_dataset = 'folhauol' 
max_inferences = 100

hyp_template = "Esse texto está relacionado a {}."

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)
classes_list = list(raw_data[class_col].unique())

In [19]:
Dataset.from_dict(raw_data)

ArrowInvalid: Could not convert 'ilustrissima' with type str: tried to convert to double

In [11]:
zeroshot_method = "dotproduct"  

zeroshot_config = {
    'similarity_model' : 'sentence-transformers/stsb-xlm-r-multilingual',
    'setfit_model' : 'sentence-transformers/stsb-xlm-r-multilingual',# 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2'
    'dataset':which_dataset,
    'class_col':class_col,
    'data_col':data_col,
    'split':"zeroshot",
    'labeling_method':zeroshot_method,
    'max_inferences':max_inferences,
    'classes':classes_list,
    'template': hyp_template,
    'random_state':422,
    'top_n': 8,
    'training_examples': 8,
    "batch_size" : 4,
    "num_pairs" : 5,
    "num_epochs" : 1,
    'st_train_epochs': 10,
    'st_train_batch_size': 20,
    # Unfreeze the head and freeze the body -> head-only training
    'keep_body_frozen_setfit': True,
    # Unfreeze the head and unfreeze the body -> end-to-end training
    # zeroshot_config['keep_body_frozen_setfit'] = False
}

data_to_label = raw_data.sample(zeroshot_config['max_inferences'],random_state=zeroshot_config['random_state']).sort_index()

model = ZeroBERTo(classes_list=zeroshot_config['classes'],labeling_dataset=data_to_label,
                embeddingModel=zeroshot_config['similarity_model'],config=zeroshot_config)
                  
model.runLabeling()
model.evaluateLabeling()

model.getLabelingMetrics()
model.saveLabelingResults()


KeyboardInterrupt: 

In [ ]:
### load results from labeling step and create training data for contrastive learning
zeroshot_previous_data = datasets_handler.getZeroshotPreviousData(
    model.config['dataset'],model.config['class_col'],top_n=model.config['top_n'],exec_time=model.config['exec_time'])

In [ ]:
df_train = datasets_handler.splitDataset(model.labeling_dataset,model.config) ### df_test will not be used

In [ ]:
# raw_data_final, model.config['new_class_col'] = datasets_handler.mergeLabelingToDataset(
#     raw_data,zeroshot_previous_data,model.config['class_col'])
# df_train['']
train_data = datasets_handler.buildDatasetDict(df_train)

In [8]:
### overrides config for looping second part of the model
# model.config['num_epochs'] = 5
# model.config['num_pairs'] = 20
# model.config['keep_body_frozen_setfit'] = False

### build trainer for contrastive learning
model.buildTrainer(train_data)

### train
model.contrastive_train()
gc.collect()

### predict labeling dataset with new classifier
model.getPredictions()
# 5000 sentences --- 4min40s ---sts-xlm-rr
# 5000 sentences ---  ---ricardo-filho/bert-portuguese

setfit_all_metrics = evaluation_metrics.get_metrics(model.y_pred, model.trainer.eval_dataset["class_code"])
print(setfit_all_metrics)

setfit_exec_time  = evaluation_metrics.saveResults(model.config,setfit_all_metrics)
print(model.config)

Applying column mapping to training dataset
The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 128.


<class 'dict'>


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running predictions on 1000 sentences.


/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'weighted': [{'accuracy': 0.036}, {'precision': 0.14999167194571073}, {'recall': 0.036}, {'f1': 0.05569551927793277}], 'macro': [{'accuracy': 0.036}, {'precision': 0.06193184719617203}, {'recall': 0.029464861379754997}, {'f1': 0.025705939817275425}]}
metrics_setfit_2023_04_08__22_15_09.csv
config_setfit_2023_04_08__22_15_09.csv
{'similarity_model': 'sentence-transformers/stsb-xlm-r-multilingual', 'setfit_model': 'sentence-transformers/stsb-xlm-r-multilingual', 'dataset': 'folhauol', 'class_col': 'class', 'data_col': 'text', 'split': 'zeroshot', 'labeling_method': 'dotproduct', 'max_inferences': 1000, 'classes': ['Poder e Política no Brasil', 'Mercado', 'Notícias de fora do Brasil', 'Esporte', 'Tecnologia', 'Meio Ambiente', 'Equilíbrio e Saúde', 'Educação', 'TV, Televisão e Entretenimento', 'Ciência', 'Turismo', 'Comida'], 'template': 'Esse texto está relacionado a {}.', 'random_state': 422, 'top_n': 8, 'n_examples': 8, 'batch_size': 4, 'num_pairs': 5, 'num_epochs': 1, 'st_train_epochs

In [6]:
model.y_pred

NameError: name 'model' is not defined